Create .env file with the following vars

```
OPENAI_API_TYPE = azure
OPENAI_API_VERSION = 2023-03-15-preview
OPENAI_API_BASE = ' ... ' # Replace with the URL of an Azure OpenAI gpt-3.5-turbo deployment
OPENAI_API_KEY = ' ... ' # Replace with the corresponding API key

```

In [50]:

# Install required packages
import sys

#!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} -c conda-forge langchain
!conda install --yes --prefix {sys.prefix} python-dotenv
!conda install --yes --prefix {sys.prefix} -c conda-forge openai 
!conda install --yes --prefix {sys.prefix} -c conda-forge tiktoken
!conda install --yes --prefix {sys.prefix} -c conda-forge pypdf
!conda install --yes --prefix {sys.prefix} -c conda-forge chromadb

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/martin/anaconda3

  added / updated specs:
    - python-dotenv


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2022.12.~ --> pkgs/main::ca-certificates-2023.01.10-h06a4308_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2022.12.7~ --> pkgs/main/linux-64::certifi-2022.12.7-py310h06a4308_0 
  conda              conda-forge::conda-23.3.1-py310hff520~ --> pkgs/main::conda-23.3.1-py310h06a4308_0 
  openssl            conda-forge::openssl-1.1.1t-h0b41bf4_0 --> pkgs/main::openssl-1.1.1t-h7f8727e_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/martin/anaconda3

  added / updated specs:
    - openai


The foll

In [51]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader # for loading the pdf

load_dotenv(dotenv_path="/.env")

loader = PyPDFLoader('power-platform-2023-release-wave-1-plan.pdf')

# combine all text
pages = loader.load_and_split()

embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

vectordb = Chroma.from_documents(pages, embedding=embeddings,
                                 persist_directory="./vectordb")

#vectordb.persist()




Using embedded DuckDB with persistence: data will be stored in: ./vectordb


In [52]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import AzureChatOpenAI


llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0.9)

pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever())


In [53]:
query = "When can we expect Microsoft Dataverse to go in Public preview?"

result = pdf_qa({"question": query, "chat_history": ""})

print("Answer:")
print(result["answer"])


Answer:
I'm sorry, but as an AI language model, I don't have access to real-time information about Microsoft Dataverse. For the latest information on its release status, I recommend checking Microsoft's official website or forums.


In [48]:
query = "List all of the features going into public preview in May 2023?"

result = pdf_qa({"question": query, "chat_history": ""})

print("Answer:")
print(result["answer"])

Answer:
According to the context, the "Test Power Apps in CICD pipelines" feature for Power Apps is planned to be enabled for public preview in May 2023. There are no other features mentioned to be going into public preview in May 2023 in the given context.


In [49]:
query = "List all of the features going into public preview in Sep 2023?"

result = pdf_qa({"question": query, "chat_history": ""})

print("Answer:")
print(result["answer"])

Answer:
According to the context, there is no feature going into public preview in September 2023 for Power Apps, Power BI, or Power Automate. The features going into general availability in September 2023 are "Control visual formatting with improved capabilities" for Power BI and "Drag and drop to build responsive pages more easily" for Power Apps.
